In [0]:
#Importing all required libraries

from __future__ import print_function, division
import os
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets, models, transforms
from torch.utils import data
from torchvision import transforms, utils
from PIL import Image
import time
import copy
from tqdm import tqdm_notebook as tqdm


# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

# Importing the data

In [2]:
# Code for extracting and loading images from Google Drive to Google Colab

use_colab = True

if use_colab == True:
    
    !pip install pydrive
    
    from google.colab import drive
    import os
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

    drive.mount('/content/drive/')
    
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    
    # Test
    #https://drive.google.com/open?id=1wzOJPY7FBZoEHyWMAd89cL9cAYwH7nP4
    # Train
    #https://drive.google.com/open?id=15zC2Qx243_TnCZA8-Aw5kN-7MwOzNghc
    # Labels
    # https://drive.google.com/open?id=1cM_BusmHKqEla-c7LKh6L7kxCi-HGjxM

    download = drive.CreateFile({'id': '1wzOJPY7FBZoEHyWMAd89cL9cAYwH7nP4'})
    download.GetContentFile('train_data.tar.gz')

    download = drive.CreateFile({'id': '1Mj-0XmAe6SrstwG7ixOJVoiX6yg-udOX'})
    download.GetContentFile('test_data.tar.gz')

    download = drive.CreateFile({'id': '1cM_BusmHKqEla-c7LKh6L7kxCi-HGjxM'})
    download.GetContentFile('train.csv')
    
    !tar -xvf train_data.tar.gz
    !tar -xvf test_data.tar.gz

Streaming output truncated to the last 5000 lines.
test_data/35001.jpg
test_data/35002.jpg
test_data/35003.jpg
test_data/35004.jpg
test_data/35005.jpg
test_data/35006.jpg
test_data/35007.jpg
test_data/35008.jpg
test_data/35009.jpg
test_data/35010.jpg
test_data/35011.jpg
test_data/35012.jpg
test_data/35013.jpg
test_data/35014.jpg
test_data/35015.jpg
test_data/35016.jpg
test_data/35017.jpg
test_data/35018.jpg
test_data/35019.jpg
test_data/35020.jpg
test_data/35021.jpg
test_data/35022.jpg
test_data/35023.jpg
test_data/35024.jpg
test_data/35025.jpg
test_data/35026.jpg
test_data/35027.jpg
test_data/35028.jpg
test_data/35029.jpg
test_data/35030.jpg
test_data/35031.jpg
test_data/35032.jpg
test_data/35033.jpg
test_data/35034.jpg
test_data/35035.jpg
test_data/35036.jpg
test_data/35037.jpg
test_data/35038.jpg
test_data/35039.jpg
test_data/35040.jpg
test_data/35041.jpg
test_data/35042.jpg
test_data/35043.jpg
test_data/35044.jpg
test_data/35045.jpg
test_data/35046.jpg
test_data/35047.jpg
test_data

# Create custom class

In [0]:
class final(Dataset):

    def __init__(self, df, root_dir, transform=None,test = False):
        
        self.df = df
        self.root_dir = root_dir
        self.transform = transform
        self.test = test

    def __len__(self):
        return len(self.df.index)

    def __getitem__(self, idx):                 # returns images with the labels 
        img_name = os.path.join(self.root_dir, self.df.iloc[idx, 0])
        img = Image.open(img_name)
        
        if not self.test: #if the dataset is testing set, then it would not have labels
            img_labels = self.df.iloc[idx, 1]
            label_tensor = torch.zeros((1, 20))
            for label in img_labels.split(" "):
                label_tensor[0, int(label)] = 1

            img_label = torch.tensor(label_tensor,dtype= torch.float32)

            if self.transform:
              img = self.transform(img)
            return (img, img_label.squeeze()) #returning images with their labels
        
        
        if self.transform:
              img = self.transform(img)
        return (img) #returning just the images (in case of testing set)


# create transforms

In [0]:
# inspired from pytorch doc: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Create the dataset

# Divide into train and val

In [14]:
dataframe = pd.read_csv('./train.csv', usecols = ['ImageID','Labels']) #reading dataset and including only first two columns
# if you want to run on less data to quickly check
#dataframe = pd.read_csv(train_csv).head(5000)

import sklearn
from sklearn.model_selection import train_test_split

train_df,val_df = train_test_split(dataframe, test_size=0.20) #splitting into test and train dataset
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

print(f"Validation_Data Length: {len(val_df)}\n Train_Data Length: {len(train_df)}")

Validation_Data Length: 6000
 Train_Data Length: 24000


In [0]:
# Train dataset
training_set = final(train_df, root_dir='./train_data/',transform=data_transforms['train'],test = False)
training_loader = data.DataLoader(dataset=training_set,batch_size=128,shuffle=False)

# validation dataset
validate_set = final(val_df, root_dir='./train_data/',transform=data_transforms['train'],test = False)
validate_loader = data.DataLoader(dataset=validate_set,batch_size=128,shuffle=False)


In [19]:
print("Size of Training set is:",len(training_set))
print("Size of Validation set is:",len(validate_set))

Size of Training set is: 24000
Size of Validation set is: 6000


In [20]:
#Displaying shapes of features and labels of each set 

train_features, train_labels = next(iter(training_loader))
print(f'Train Features: {train_features.shape}\nTrain Labels: {train_labels.shape}')
print()
val_features, val_labels = next(iter(validate_loader))
print(f'Validation Features: {val_features.shape}\nValidation Labels: {val_labels.shape}')
print()

Train Features: torch.Size([128, 3, 224, 224])
Train Labels: torch.Size([128, 20])

Validation Features: torch.Size([128, 3, 224, 224])
Validation Labels: torch.Size([128, 20])



# Loading Pre trained weights

In [0]:
# original saved file with DataParallel
state_dict = torch.load('/content/resnet_2_epochs.h5py')


# Build the model now

In [0]:
import torch.nn.functional as F

In [0]:
resnet_class = models.resnet50(pretrained=True)

class AvgPool(nn.Module):
    def forward(self, x):
        return F.avg_pool2d(x, x.shape[2:])
    
class ResNet50(nn.Module):
    def __init__(self,num_outputs):
        super(ResNet50,self).__init__()
        self.resnet = resnet_class
        layer4 = self.resnet.layer4
        self.resnet.layer4 = nn.Sequential(
                                    nn.Dropout(0.5),
                                    layer4
                                    )
        self.resnet.avgpool = AvgPool()
        self.resnet.fc = nn.Linear(2048, num_outputs)
        for param in self.resnet.parameters():
            param.requires_grad = False

        for param in self.resnet.layer4.parameters():
            param.requires_grad = True

        for param in self.resnet.fc.parameters():
            param.requires_grad = True
            
    def forward(self,x):
        out = self.resnet(x)
        return out

In [0]:
out_dimensions = 20
NeuralNet = ResNet50(num_outputs = out_dimensions) #loading pretrained network and setting output dimensions as 20

In [29]:
NeuralNet.load_state_dict(state_dict)
NeuralNet.eval() #displays configuration of the loaded neural network

ResNet50(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          

In [30]:
#count of total parameters
params_total = sum(p.numel() for p in NeuralNet.parameters())
print(f'{params_total:,} total parameters.')
trainable_params_total = sum(p.numel() for p in NeuralNet.parameters() if p.requires_grad)
print(f'{trainable_params_total:,} training parameters.')

23,549,012 total parameters.
15,005,716 training parameters.


In [31]:
BATCH_SIZE=128

print("TRAINING")
print("training examples: ",len(training_set))
print("batch size: ",BATCH_SIZE)
print("batches available: ",len(training_loader))
print()
print("VALIDATION")
print("validation examples: ",len(validate_set))
print("batch size: ",BATCH_SIZE)
print("batches available: ",len(validate_loader))
print()

TRAINING
training examples:  24000
batch size:  128
batches available:  188

VALIDATION
validation examples:  6000
batch size:  128
batches available:  47



In [0]:
#Assigning values to parameters

NUM_EPOCHS = 2
PERCENTILE = 90
LEARNING_RATE = 0.01
DISABLE_TQDM = False

In [0]:
#creating dictionary for dataloaders

dataloaders_dict = {'train':training_loader, 'val':validate_loader}

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

TRAINING THE MODEL

In [0]:
## TRAINING MODEL AND GENERATING WEIGHTS
## COMMENTED OUT BECAUSE WEIGHTS ARE PRE LOADED
## UNCOMMENT IF YOU WANT TO FRESHLY TRAIN THE SET (Takes time)


# optimizer = optim.Adam(NeuralNet.parameters(),lr = LEARNING_RATE)
# loss_func = torch.nn.BCEWithLogitsLoss()
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,patience = 2)
# best_loss = np.inf
# for epoch in range(NUM_EPOCHS):
#     for phase in ['train', 'val']:
#         start_time = time.time()
#         if phase == 'train':
#             NeuralNet.train()
#         else:
#             NeuralNet.eval()
            
#         running_loss = 0.0
#         for images_batch, labels_batch in tqdm(dataloaders_dict[phase],disable = DISABLE_TQDM):
#             images_batch = images_batch.to(device)
#             labels_batch = labels_batch.to(device)
            
#             optimizer.zero_grad()
            
#             with torch.set_grad_enabled(phase == 'train'):
#                 pred_batch = NeuralNet(images_batch)
#                 loss = loss_func(pred_batch,labels_batch)
                
#             if phase == 'train':
#                 loss.backward()
#                 optimizer.step()
                
#             running_loss += loss.item() * images_batch.size(0)    
#         epoch_loss = running_loss / len(dataloaders_dict[phase].dataset)            

#         if phase == 'val' and epoch_loss < best_loss:            
#             print("model val_loss Improved from {:.8f} to {:.8f}".format(best_loss,epoch_loss))
#             best_loss = epoch_loss
#             best_model_wts = copy.deepcopy(NeuralNet.state_dict())
        
#         if phase == 'val':
#             scheduler.step(epoch_loss)
        
#         elapsed_time = time.time()-start_time
#         print("Phase: {} | Epoch: {}/{} | {}_loss:{:.8f} | Time: {:.4f}s".format(phase,
#                                                                               epoch+1,
#                                                                               NUM_EPOCHS,
#                                                                               phase,
#                                                                               epoch_loss,
#                                                                               elapsed_time))
# NeuralNet.load_state_dict(best_model_wts)

# checking on validation data

# Testing on Validation Set

In [38]:
len(validate_set)

6000

In [45]:
#using this function for val
val_preds = np.zeros((len(validate_set), out_dimensions))
i = 0
#features_val, labels = next(iter(val_loader))
for batch_test in tqdm(validate_loader,disable = DISABLE_TQDM):
    batch_test = batch_test[0].to(device)
    batch_preds = NeuralNet(batch_test).detach().cpu().numpy()
    val_preds[i * BATCH_SIZE:(i+1) * BATCH_SIZE, :] = batch_preds
    i+=1

In [0]:
predicted_values = []
for i in range(len(val_preds)):         
    idx = np.where(val_preds[i] > np.percentile(val_preds[i],PERCENTILE))    
    predicted_values.append(idx[0])

In [0]:
val_labels = val_df['Labels']

# checking score

In [48]:
from sklearn.metrics import recall_score,precision_score,f1_score

#functions to convert into one hot labels

def convert_to_ohe_list(y): #takes y as a list of lists (for predicted labels)
  x=[]
  for arr in y:
    temp=[]
    for i in range(20):
      temp.append(0)
    for ele in arr:
      # print(ele)
      temp[int(ele)]=1
    x.append(temp)
  return x

def convert_to_ohe_string(y): #takes y as a list of strings (for actual labels)
  x=[]
  for arr in y:
    a=arr.split()
    temp=[]
    for i in range(20):
      temp.append(0)
    for ele in a:
      # print(ele)
      temp[int(ele)]=1
    x.append(temp)
  return x

y_true=convert_to_ohe_string(val_labels) #converting into one hot labels for comparison with predicted set
y_pred=convert_to_ohe_list(predicted_values) #converting into one hot labels for comparison with true set

print("Recall Score for validation set is:",recall_score(y_true=y_true, y_pred=y_pred, average='samples'))
print("Precision Score for validation set is:",precision_score(y_true=y_true, y_pred=y_pred, average='samples'))
print("F1 Score for validation set is:",f1_score(y_true=y_true, y_pred=y_pred, average='samples'))

Recall Score for validation set is: 0.7949706349206349
Precision Score for validation set is: 0.5489166666666667
F1 Score for validation set is: 0.6170988095238096


#Testing set now

In [0]:
# test dataset
test_df = pd.read_csv('./test.csv', usecols = ['ImageID'])
# Test dataset/loader
test_dataset = final(test_df, root_dir='./test_data/',transform=data_transforms['train'],test = True)
test_loader = data.DataLoader(dataset=test_dataset,batch_size=128,shuffle=False)


## generating test predictions

In [0]:
test_preds = np.zeros((len(test_dataset), out_dimensions))
i = 0
for test_batch in tqdm(test_loader,disable = DISABLE_TQDM):
    test_batch = test_batch.to(device)
    batch_preds = NeuralNet(test_batch).detach().cpu().numpy()
    test_preds[i * BATCH_SIZE:(i+1) * BATCH_SIZE, :] = batch_preds
    i+=1

## creating submission

In [0]:
PERCENTILE = 90

In [0]:
preds = []
for i in range(len(test_preds)):         
    idx = np.where(test_preds[i] > np.percentile(test_preds[i],PERCENTILE))    
    preds.append(idx[0])

In [0]:
test_df['predicted_labels'] = preds
test_df['predicted_labels'] = test_df['predicted_labels'].apply(lambda x : ' '.join(map(str,list(x))))
test_df = test_df.rename(columns={0: 'id'})
#test_df['id'] = test_df['id'].apply(lambda x : x.split('.')[0])
test_df.head()

,ImageID,predicted_labels
0,30000.jpg,1 17
1,30001.jpg,1 15
2,30002.jpg,1 15
3,30003.jpg,1 16
4,30004.jpg,1 15


In [0]:
test_df.to_csv('./submission.csv',index = False)